In [190]:
import pandas as pd
import numpy as np
import re
import string
import jieba
import collections
import codecs

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [191]:
# all_question = pd.read_csv('./data/description.csv')
# all_question = pd.read_excel('./data/Herceptin.xlsx')
# all_question = pd.read_excel('./data/Herceptin_funding_filtered.xlsx')
all_question = pd.read_excel('./data/Herceptin_patient_filtered.xlsx')

In [192]:
all_question.head(10)

,Case Number,Contact Type,Contact Channel,Response Channel,Enquiry Classification,Subject,Description,Product,Response,Unnamed: 9
0,1420656,Nurse,Affiliate MI mailbox,Phone,Stability/Storage/Reconstitution,beyond temperature,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配...",Herceptin,根据说明书信息，赫赛汀需要存放于2-8度的温度中，超温保存对于药品活性会产生影响，不建议使用。,NaN
1,1423956,Internal,Phone,Email,Stability/Storage/Reconstitution,beyond temperature-herceptin,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）,Herceptin,[#Responded via email: 罗氏医学信息: [01423956]#],NaN
3,1434184,Physician,Internal Webform,Email,Safety,AE-Herceptin&Thromboembolism,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）,Herceptin,[#Responded via email: 罗氏医学信息: [01434184]#],NaN
5,1438071,Internal,Phone,Email;Phone,Product Quality,quality-herceptin,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用,Herceptin,根据Herceptin说明书信息，溶解后为无色至淡黄色，澄清至微乳光溶液。该case记录PC上报,NaN
6,1439604,Internal,In person,Email,Publication/Educational Material Request,PI-herceptin,赫赛汀最新说明书,Herceptin,[#Responded via email: 罗氏医学信息: [01439604]#],NaN
9,1444557,Nurse,Phone,Phone,Stability/Storage/Reconstitution,beyond temperature-Herceptin,赫赛汀注射前发现稀释液有结冰情况，能否使用，产品批号（SH0128，N3834/B3051）,Herceptin,根据说明书信息，赫赛汀需要在2-8度保存和运输，超温保存会影响药品活性，从而影响疗效和安全性...,NaN
14,1444893,Nurse,Internal Webform,Email,Dosage/Administration,administration-Herceptin,[Wechat] 如果不同批号的药品基于追溯不良反应考虑分开使用，请问所需盐水的量和注射时间...,Herceptin,[#Responded via email: 罗氏医学信息: [01444893]#],NaN
15,1446436,Internal,Phone,Email,Safety,AE-herceptin&Extravasation,赫赛汀如果外渗对外周组织会有何影响（确认无AE）,Herceptin,[#Responded via email: 罗氏医学信息: [01446436]#],NaN
16,1453717,Wholesaler,Affiliate MI mailbox,Phone,Logistics/Funding/Regulatory,supply-herceptin,有医院的肿瘤科需要药品赫赛汀，想了解在医院销售的做法。,Herceptin,患者热线无法提供商业销售方面的信息，请与当地商务联系,NaN
17,1453767,Physician,Internal Webform,Email,Safety,AE-Herceptin&psoriasis,[Wechat] 使用赫赛汀对银屑病是否有影响？,Herceptin,[#Responded via email: 罗氏医学信息: [01453767]#],NaN


In [193]:
all_question.dropna(axis=0, inplace=True, subset=['Description'])

In [194]:
all_question.head(10)

,Case Number,Contact Type,Contact Channel,Response Channel,Enquiry Classification,Subject,Description,Product,Response,Unnamed: 9
0,1420656,Nurse,Affiliate MI mailbox,Phone,Stability/Storage/Reconstitution,beyond temperature,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配...",Herceptin,根据说明书信息，赫赛汀需要存放于2-8度的温度中，超温保存对于药品活性会产生影响，不建议使用。,NaN
1,1423956,Internal,Phone,Email,Stability/Storage/Reconstitution,beyond temperature-herceptin,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）,Herceptin,[#Responded via email: 罗氏医学信息: [01423956]#],NaN
3,1434184,Physician,Internal Webform,Email,Safety,AE-Herceptin&Thromboembolism,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）,Herceptin,[#Responded via email: 罗氏医学信息: [01434184]#],NaN
5,1438071,Internal,Phone,Email;Phone,Product Quality,quality-herceptin,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用,Herceptin,根据Herceptin说明书信息，溶解后为无色至淡黄色，澄清至微乳光溶液。该case记录PC上报,NaN
6,1439604,Internal,In person,Email,Publication/Educational Material Request,PI-herceptin,赫赛汀最新说明书,Herceptin,[#Responded via email: 罗氏医学信息: [01439604]#],NaN
9,1444557,Nurse,Phone,Phone,Stability/Storage/Reconstitution,beyond temperature-Herceptin,赫赛汀注射前发现稀释液有结冰情况，能否使用，产品批号（SH0128，N3834/B3051）,Herceptin,根据说明书信息，赫赛汀需要在2-8度保存和运输，超温保存会影响药品活性，从而影响疗效和安全性...,NaN
14,1444893,Nurse,Internal Webform,Email,Dosage/Administration,administration-Herceptin,[Wechat] 如果不同批号的药品基于追溯不良反应考虑分开使用，请问所需盐水的量和注射时间...,Herceptin,[#Responded via email: 罗氏医学信息: [01444893]#],NaN
15,1446436,Internal,Phone,Email,Safety,AE-herceptin&Extravasation,赫赛汀如果外渗对外周组织会有何影响（确认无AE）,Herceptin,[#Responded via email: 罗氏医学信息: [01446436]#],NaN
16,1453717,Wholesaler,Affiliate MI mailbox,Phone,Logistics/Funding/Regulatory,supply-herceptin,有医院的肿瘤科需要药品赫赛汀，想了解在医院销售的做法。,Herceptin,患者热线无法提供商业销售方面的信息，请与当地商务联系,NaN
17,1453767,Physician,Internal Webform,Email,Safety,AE-Herceptin&psoriasis,[Wechat] 使用赫赛汀对银屑病是否有影响？,Herceptin,[#Responded via email: 罗氏医学信息: [01453767]#],NaN


In [195]:
all_question.isna().sum()

Case Number                 0
Contact Type                0
Contact Channel             0
Response Channel            0
Enquiry Classification      0
Subject                     0
Description                 0
Product                     0
Response                    0
Unnamed: 9                201
dtype: int64

In [196]:
all_source_data = []
for idx,row in all_question.iterrows():
    all_source_data.append(row['Description'])
len(all_source_data)

201

In [198]:
# 去除所有“[]”内的文本
def remove_special_string(text):
#     return text.replace("[Wechat]", "")
    text = re.sub(r'\[[\s\S]*\]', '', text)
    text = re.sub(r'（[\s\S]*）', '', text)
    text = re.sub(r'\<[\s\S]*\>', '', text)
    text = re.sub(r'[0-9]*', '', text)
    return text
def remove_stopwords(text, stopwords):
    text_cutted = jieba.cut(text, cut_all=False)
    text_removed = []
    for word in text_cutted:
        if word in stopwords or word.strip()=="":
            continue
        text_removed.append(word)
    return text_removed

In [199]:
all_question['Description'] = all_question.Description.apply(lambda x: remove_special_string(x))
# all_question['Description'] = all_question.Description.apply(lambda x: remove_stopwords(x, stopwords))

In [200]:
# all_question.to_excel('./data/after_re_sub.xlsx')
all_question.head()

,Case Number,Contact Type,Contact Channel,Response Channel,Enquiry Classification,Subject,Description,Product,Response,Unnamed: 9
0,1420656,Nurse,Affiliate MI mailbox,Phone,Stability/Storage/Reconstitution,beyond temperature,赫赛汀 配了之后结冰了， 能否继续使用。,Herceptin,根据说明书信息，赫赛汀需要存放于2-8度的温度中，超温保存对于药品活性会产生影响，不建议使用。,NaN
1,1423956,Internal,Phone,Email,Stability/Storage/Reconstitution,beyond temperature-herceptin,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用,Herceptin,[#Responded via email: 罗氏医学信息: [01423956]#],NaN
3,1434184,Physician,Internal Webform,Email,Safety,AE-Herceptin&Thromboembolism,请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献,Herceptin,[#Responded via email: 罗氏医学信息: [01434184]#],NaN
5,1438071,Internal,Phone,Email;Phone,Product Quality,quality-herceptin,常规配置和保存的赫赛汀复溶后的溶液保存在-度的情况下，现出现浑浊情况，能否使用,Herceptin,根据Herceptin说明书信息，溶解后为无色至淡黄色，澄清至微乳光溶液。该case记录PC上报,NaN
6,1439604,Internal,In person,Email,Publication/Educational Material Request,PI-herceptin,赫赛汀最新说明书,Herceptin,[#Responded via email: 罗氏医学信息: [01439604]#],NaN


In [95]:
jieba.load_userdict('./jieba_dict.txt')

In [201]:
demo_wordcut = jieba.cut('赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。', cut_all=False)
print(' '.join(demo_wordcut).split())

['赫赛汀', '（', '批号', '1', '：', 'N3825B03B3049', ',', '批号', '2', '：', 'N3826B02B3050', '）', '配', '了', '之后', '结冰', '了', '，', '能否', '继续', '使用', '。']


In [202]:
len(all_question)

201

In [203]:
all_text_data = []
for idx,row in all_question.iterrows():
    all_text_data.append(row['Description'])
len(all_text_data)

201

In [99]:
len(all_text_data)

757

In [205]:
chinese_punc = string.punctuation + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."""
text = "赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。"
text.translate(str.maketrans('', '', chinese_punc))

'赫赛汀 批号1\xa0N3825B03B3049 批号2 N3826B02B3050 配了之后结冰了 能否继续使用'

In [206]:
chinese_punc = string.punctuation + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."""
def jieba_cut(text):
    text = text.translate(str.maketrans('', '', chinese_punc))
    text = remove_stopwords(text, stopwords)
    return text

In [207]:
# term-tfidf matrix
tfidf_vectorizer = TfidfVectorizer(input='content', tokenizer=jieba_cut, use_idf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(all_text_data)
tfidf_matrix

<201x1212 sparse matrix of type '<class 'numpy.float64'>'
	with 2407 stored elements in Compressed Sparse Row format>

In [107]:
tfidf_matrix

<757x1998 sparse matrix of type '<class 'numpy.float64'>'
	with 9954 stored elements in Compressed Sparse Row format>

In [208]:
'''Latent Semantic Analysis'''
lsa = TruncatedSVD(n_components=100, n_iter=10)
idf_lsa_vector = lsa.fit_transform(tfidf_matrix)
idf_lsa_vector.shape

(201, 100)

In [109]:
type(idf_lsa_vector)
idf_lsa_vector.shape

(757, 100)

In [229]:
n_clusters=50
max_iter = 300
kmeans_model = KMeans(n_clusters=n_clusters, max_iter=max_iter)
# kmeans_model.fit(tfidf_matrix)
kmeans_model.fit(idf_lsa_vector)
# kmeans_model.fit(doc2vec_matrix)
kmeans_clustering = collections.defaultdict(list)
for idx, label in enumerate(kmeans_model.labels_):
    kmeans_clustering[label].append(idx)

In [230]:
f = open('./patient_filtered_kmeans_'+str(n_clusters)+'class_'+str(max_iter)+'iter_demo'+'.txt', 'w')
for key, value in kmeans_clustering.items():
    for idx in value:
#         print(all_source_data[idx])
        f.write(all_source_data[idx] + '\n')
#         f.write(all_text_data[idx] + '\n')
    f.write('\n-------------------------------------------------------------------------------------------------------------\n\n')
f.close()

# for idx in kmeans_clustering[30]:
#     print(all_source_data[idx])

In [339]:
dbscan_model = DBSCAN(eps=0.01, min_samples=10)
dbscan_model.fit(tfidf_matrix)
dbscan_clustering = collections.defaultdict(list)
for idx, label in enumerate(dbscan_model.labels_):
    dbscan_clustering[label].append(idx)

In [342]:
print(len(set(dbscan_clustering)))
for idx in dbscan_clustering[2]:
    print(all_text_data[idx])

3
